<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Enforce Mandatory Tags Across All AWS Resources using unSkript legos.</b>
</div>

<br>

<center><h2>Enforce Mandatory Tags Across All AWS Resources</h2></center>

# Steps Overview
   1. List all the Untagged Resources ARNs in the given region.
   2. Get tag keys of all Resources.
   3. Attach Mandatory Tags to All the AWS Resources.

Here we will use unSkript AWS Get Untagged Resources Lego. This lego take region: str as input. This inputs is used to find out all Untagged Resources.

In [2]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_untagged_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_untagged_resources(handle, region: str) -> List:
    """aws_get_untagged_resources Returns an List of Untagged Resources.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: Region to filter resources.

        :rtype: List of untagged resources.
    """

    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = []
    try:
        response = aws_get_paginator(ec2Client, "get_resources", "ResourceTagMappingList")
        for resources in response:
            if not resources["Tags"]:
               result.append(resources["ResourceARN"])
    except Exception as error:
        result.append({"error":error})

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="UntaggedResources")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_untagged_resources, lego_printer=aws_get_untagged_resources_printer, hdl=hdl, args=args)

Here we will use unSkript Get Tag Keys Of All Resources Lego. This lego take region: str as input. This input is used to find out all Tag Keys of Resources.

In [2]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_resources_tags_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_resources_tags(handle, region: str) -> List:
    """aws_resources_tags Returns an List of all Resources Tags.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: Region to filter resources.

        :rtype: List of all Resources Tags.
    """
    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = []
    try:
        response = aws_get_paginator(ec2Client, "get_tag_keys", "TagKeys")
        result = response
    except Exception as error:
        result.append({"error":error})

    return result



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_resources_tags, lego_printer=aws_resources_tags_printer, hdl=hdl, args=args)

Here we will use unSkript AWS Attach Tags to Resources Lego. This lego take handle, resource_arn: list, tag_key: str, tag_value: str, region: str as input. This input is used to attach mandatory tags to all untagged Resources.

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_tag_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_tag_resources(handle, resource_arn: list, tag_key: str, tag_value: str, region: str) -> Dict:
    """aws_tag_resources Returns an Dict of resource info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type resource_arn: list
        :param resource_arn: Resource ARNs.

        :type tag_key: str
        :param tag_key: Resource Tag Key.

        :type tag_value: str
        :param tag_value: Resource Tag value.

        :type region: str
        :param region: Region to filter resources.

        :rtype: Dict of resource info.
    """
    ec2Client = handle.client('resourcegroupstaggingapi', region_name=region)
    result = {}
    try:
        response = ec2Client.tag_resources(
            ResourceARNList=resource_arn,
            Tags={tag_key: tag_value}
            )
        result = response

    except Exception as error:
        result["error"] = error

    return result






task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "resource_arn": "iter_item",
    "tag_key": "Tag_Key",
    "tag_value": "Tag_Value"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "UntaggedResources",
    "iter_parameter": "resource_arn"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_tag_resources, lego_printer=aws_tag_resources_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to attach tags. This Runbook gets the list of all untagged resources of a given region, discovers tag keys of the given region and attaches mandatory tags to all the untagged resource. To view the full platform capabilities of unSkript please visit https://unskript.com